In [1]:
# cwd = Partial-DANN/
import os
os.chdir("..")
print(os.getcwd())
from main import *

C:\Users\User\Desktop\Kevin\Partial-DANN


In [2]:
config = configparser.ConfigParser()
config.read("./config.ini")

# Data
root = config["path"]["root"]
output = config["path"]["output"]
debug = config["path"]["debug"]
dataset = datasets["simple_amos"]
modality = "ct+mr:balance"
masked = True
train_data = "split"
mode = "train"

# Module
module_name = "co_training"
pretrained = None
batch_size = 1
loss = "dice2"
lr = 0.001
optim = "Adam"
max_iter = 10000
eval_step = 100

# Efficiency
deterministic = False
dev = False
cache_rate = 0.1
num_workers = 2

In [3]:
print("root dir:", root)
print("checkpoint dir:", output)
print("\ndataset info:")
display(dataset)

root dir: C:/Users/User/Desktop/TSM_Project/data/amos22
checkpoint dir: ./checkpoints

dataset info:


{'name': 'SIMPLE_AMOS',
 'class': medaset.amos.SimpleAMOSDataset,
 'train_transforms': <monai.transforms.compose.Compose at 0x28b2b56be80>,
 'val_transforms': <monai.transforms.compose.Compose at 0x28b2b5c1910>,
 'num_classes': 9,
 'fg': {'ct': [1, 3, 5, 7], 'mr': [2, 4, 6, 8]},
 'bg': {'ct': {2: 0, 4: 0, 6: 0, 8: 0}, 'mr': {1: 0, 3: 0, 5: 0, 7: 0}}}

In [ ]:
## Whether train without randomness
if deterministic:
    set_determinism(seed=0)
    print("** Deterministic = True")

## Dataloaders
train_dataset, val_dataset, test_dataset = get_datasets(
    dataset=dataset,
    modality=modality,
    train_data=train_data,
    masked=masked,
    return_modality_dataset=modules[module_name]["return_modality_dataset"],
    root_dir=root,
    cache_rate=cache_rate,
    num_workers=num_workers,
    dev=dev,
)
train_dataloader, val_dataloader, test_dataloader = modules[module_name]["initializer"].init_dataloaders(
    train_dataset, val_dataset, test_dataset, batch_size, dev
)

## Initialize module
module = modules[module_name]["initializer"].init_module(
    loss, optim, lr, dataset, modality, masked, device,
)
if pretrained:
    print("** Pretrained checkpoint =", pretrained)
    module.load(pretrained)
module.to(device)

## Train or test
# ** note: temp checkpoints are saved in the "debug" directory
#          to separate the result of experiments and temporary
#          checkpoints generated in developer mode.
checkpoint_dir = output if not dev else debug
# create subfolder based on time
checkpoint_dir = Path(checkpoint_dir) / datetime.now().strftime("%Y%m%d-%H%M%S")
trainer = modules[module_name]["initializer"].init_trainer(
    num_classes=dataset["num_classes"],
    max_iter=max_iter,
    eval_step=eval_step,
    checkpoint_dir=checkpoint_dir,
    device=device,
)

from torch.profiler import profile, record_function, ProfilerActivity

print("** Mode =", mode)
if mode == "train":        
    trainer.train(module, train_dataloader, val_dataloader)
    # Save command-line arguments
    Path(trainer.checkpoint_dir).mkdir(parents=True, exist_ok=True)

    # Comment the code below since there is no args
    # with open(Path(trainer.checkpoint_dir) / "json", "w") as f:
    #     json.dump(vars(args), f, indent=4)
if mode == "test" or test_dataloader:
    test_metric = trainer.validation(module, test_dataloader, label="all")
    print("** Test (Final):", test_metric)
else:
    raise ValueError("Got an invalid input of option --mode.")

** Dataset = SIMPLE_AMOS
** Modality = ct+mr:balance
** Training set = split
** Annotation masked = True
** Foreground =
  - ct: [1, 3, 5, 7] 
  - mr: [2, 4, 6, 8]


Loading dataset: 100%|███████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]

BasicUNet features: (32, 32, 64, 128, 256, 32).
BasicUNet features: (32, 32, 64, 128, 256, 32).


BasicUNet features: (32, 32, 64, 128, 256, 32).
BasicUNet features: (32, 32, 64, 128, 256, 32).
** Mode = train


  0%|          | 0/240 [00:00<?, ?it/s]

...

No improvement. Validation: (New) 0.6282263 <= (Old) 0.6299671


  0%|                                                                                           | 0/20 [00:00<…

No improvement. Validation: (New) 0.5792950 <= (Old) 0.6299671


  0%|                                                                                           | 0/20 [00:00<…

Model saved! Validation: (New) 0.6370088 > (Old) 0.6299671
